# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import json

# Import API key
from api_keys import geoapify_key

#print(geoapify_key)

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("./output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-6.24,78,39,5.30,GS,1718229049
1,1,hermanus,-34.4187,19.2345,14.19,84,0,0.94,ZA,1718229049
2,2,qarauyl,48.9451,79.2550,13.34,81,80,0.09,KZ,1718229049
3,3,tommot,58.9564,126.2925,14.49,78,97,0.26,RU,1718229049
4,4,chapaev,50.2000,51.1667,23.26,44,100,4.00,KZ,1718229049


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 0.50,
    color = "City"
)

# Display the map

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_conditions_cities_df = city_data_df[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21) & (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
# We are going to drop null values using the criteria from the requirments (max temp, windspeed, and cloudiness)

dropped_cities_df = ideal_conditions_cities_df.dropna(subset=['Max Temp', 'Wind Speed', 'Cloudiness'])

# Display sample data
dropped_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,benguela,-12.5763,13.4055,21.81,72,0,1.33,AO,1718229051
88,88,macae,-22.3708,-41.7869,24.86,61,0,4.12,BR,1718229009
143,143,arica,-18.4750,-70.3042,22.75,72,0,2.57,CL,1718228432
150,150,nova xavantina,-14.6667,-52.3500,24.34,45,0,0.49,BR,1718229055
170,170,firuzabad,28.8438,52.5707,25.43,20,0,2.47,IR,1718229055
188,188,conceicao da barra,-18.5933,-39.7322,21.49,77,0,1.96,BR,1718229056
279,279,kill devil hills,36.0307,-75.6760,24.43,64,0,4.12,US,1718229062
280,280,tartouss,34.8890,35.8866,25.57,70,0,1.48,SY,1718229062
361,361,harsin,34.2721,47.5861,21.45,37,0,0.73,IR,1718229066
374,374,marathon,44.9000,-89.7668,23.58,64,0,3.60,US,1718228969


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = dropped_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ""

# Display sample data

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
58,benguela,AO,-12.5763,13.4055,72,
88,macae,BR,-22.3708,-41.7869,61,
143,arica,CL,-18.4750,-70.3042,72,
150,nova xavantina,BR,-14.6667,-52.3500,45,
170,firuzabad,IR,28.8438,52.5707,20,
188,conceicao da barra,BR,-18.5933,-39.7322,77,
279,kill devil hills,US,36.0307,-75.6760,64,
280,tartouss,SY,34.8890,35.8866,70,
361,harsin,IR,34.2721,47.5861,37,
374,marathon,US,44.9000,-89.7668,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    # use the row from the dataframe to extract the latitude and longitude
    
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle: {latitude}, {longitude}, {radius}"
    params["bias"] = f"proximity: {latitude}, {longitude}"
    
    # Set base URL
    
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    
    name_address = name_address.json()
    
    #print(json.dumps(name_address, indent = 4))
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
benguela - nearest hotel: No hotel found
macae - nearest hotel: No hotel found
arica - nearest hotel: No hotel found
nova xavantina - nearest hotel: No hotel found
firuzabad - nearest hotel: No hotel found
conceicao da barra - nearest hotel: No hotel found
kill devil hills - nearest hotel: No hotel found
tartouss - nearest hotel: No hotel found
harsin - nearest hotel: No hotel found
marathon - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
varzea da palma - nearest hotel: No hotel found
eastham - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
58,benguela,AO,-12.5763,13.4055,72,No hotel found
88,macae,BR,-22.3708,-41.7869,61,No hotel found
143,arica,CL,-18.4750,-70.3042,72,No hotel found
150,nova xavantina,BR,-14.6667,-52.3500,45,No hotel found
170,firuzabad,IR,28.8438,52.5707,20,No hotel found
188,conceicao da barra,BR,-18.5933,-39.7322,77,No hotel found
279,kill devil hills,US,36.0307,-75.6760,64,No hotel found
280,tartouss,SY,34.8890,35.8866,70,No hotel found
361,harsin,IR,34.2721,47.5861,37,No hotel found
374,marathon,US,44.9000,-89.7668,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot

map_plot_2 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 0.25,
    color = "City",
    hover_cols = ['Hotel Name', 'City', 'Country'],
    hover_alpha = 0.5
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)